In [1]:
from math import radians, cos, sin, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
  # Radius of the earth in km
  R = 6371.0

  # Convert degrees to radians
  lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

  # Differences
  dlon = lon2 - lon1
  dlat = lat2 - lat1

  # Haversine formula
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  # Distance
  distance = R * c
  return distance

In [4]:
import pandas as pd
df = pd.read_csv('../data/completed_orders.csv')

In [7]:
from math import radians, cos, sin, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
  # Radius of the earth in km
  R = 6371.0

  # Convert degrees to radians
  lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

  # Differences
  dlon = lon2 - lon1
  dlat = lat2 - lat1

  # Haversine formula
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  # Distance
  distance = R * c
  return distance

# Split the 'Trip Origin' and 'Trip Destination' into latitude and longitude
# Split the 'Trip Origin' and 'Trip Destination' into latitude and longitude
df[['Origin_lat', 'Origin_lon']] = df['Trip Origin'].str.split(',', expand=True)
df[['Dest_lat', 'Dest_lon']] = df['Trip Destination'].str.split(',', expand=True)

# Convert to float
for col in ['Origin_lat', 'Origin_lon', 'Dest_lat', 'Dest_lon']:
  df[col] = df[col].astype(float)

# Convert 'Trip Start Time' and 'Trip End Time' to datetime
df['Trip Start Time'] = pd.to_datetime(df['Trip Start Time'])
df['Trip End Time'] = pd.to_datetime(df['Trip End Time'])

# Calculate the distance
df['Distance'] = df.apply(lambda row: calculate_distance(row['Origin_lat'], row['Origin_lon'], row['Dest_lat'], row['Dest_lon']), axis=1)

# Calculate the time difference in hours
df['Trip Duration'] = (df['Trip End Time'] - df['Trip Start Time']).dt.total_seconds() / 3600

# Calculate the speed
df['Speed'] = df['Distance'] / df['Trip Duration']

In [8]:
# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| Trip ID   | Trip Origin                        | Trip Destination                | Trip Start Time     | Trip End Time       | Origin_lat   | Origin_lon   | Dest_lat   | Dest_lon   | Distance   | Trip Duration   | Speed   |
|:----------|:-----------------------------------|:--------------------------------|:--------------------|:--------------------|:-------------|:-------------|:-----------|:-----------|:-----------|:----------------|:--------|
| 391996    | 6.508813001668548,3.37740316890347 | 6.650969799999999,3.3450307     | 2021-07-01 07:28:04 | 2021-07-01 07:29:37 | 6.50881      | 3.3774       | 6.65097    | 3.34503    | 16.2065    | 0.0258333       | 627.35  |
| 391997    | 6.4316714,3.4555375                | 6.4280814653326,3.4721885847586 | 2021-07-01 06:38:04 | 2021-07-01 07:07:28 | 6.43167      | 3.45554      | 6.42808    | 3.47219    | 1.88268    | 0.49            | 3.84219 |
| 391998    | 6.631679399999999,3.3388976        | 6.508324099999999,3.3590397     | 2021-07-01 

In [10]:
# Get the number of rows where 'Speed' is greater than 60
num_fast = len(df[df['Speed'] > 60])

# Calculate the percentage
percentage_fast = (num_fast / len(df)) * 100

# Print the results
print(f"Number of trips with speed > 60 km/hr: {num_fast}")
print(f"Percentage of trips with speed > 60 km/hr: {percentage_fast}%")

Number of trips with speed > 60 km/hr: 24984
Percentage of trips with speed > 60 km/hr: 4.661020111189881%
